In [32]:
#import sys
#sys.path.insert(1, '../RTFM')
# go to RTFM main folder and run 'pip install -e .', then rtfm, model and core should be available as packages
from rtfm import featurizer as X
from rtfm import tasks # needed to make rtfm visible as Gym env
from core import environment # env wrapper
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import copy

In [14]:
# simulate flags without argparse
class Flags():
    def __init__(self, 
                 #env="rtfm:groups_simple_stationary-v0", # simplest env
                 env="rtfm:groups-v0",
                 height=5,
                 width=5,
                 partial_observability=False,
                 max_placement=2,
                 shuffle_wiki=True,
                 time_penalty=-0.02
                 ):
        self.env = env
        self.height = height
        self.width = width
        self.partial_observability = partial_observability
        self.max_placement = max_placement
        self.shuffle_wiki = shuffle_wiki 
        self.time_penalty = time_penalty

In [15]:
flags = Flags()

In [16]:
def create_env(flags, featurizer=None):
    f = featurizer or X.Concat([X.Text(), X.ValidMoves()])
    env = gym.make(flags.env, 
                   room_shape=(flags.height, flags.width), 
                   partially_observable=flags.partial_observability, 
                   max_placement=flags.max_placement, 
                   featurizer=f, 
                   shuffle_wiki=flags.shuffle_wiki, 
                   time_penalty=flags.time_penalty)
    return env

In [18]:
for i in range(100):
    print(i)
    gym_env = create_env(flags)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
x = np.array([1])
x[0]

1

In [7]:
gym_env.step(int(x[0]))

({'name': tensor([[[[  3,   0,   0,   0,   0,   0,   0,   0],
            [170,   0,   0,   0,   0,   0,   0,   0]],
  
           [[  3,   0,   0,   0,   0,   0,   0,   0],
            [170,   0,   0,   0,   0,   0,   0,   0]],
  
           [[  3,   0,   0,   0,   0,   0,   0,   0],
            [170,   0,   0,   0,   0,   0,   0,   0]],
  
           [[  3,   0,   0,   0,   0,   0,   0,   0],
            [170,   0,   0,   0,   0,   0,   0,   0]],
  
           [[  3,   0,   0,   0,   0,   0,   0,   0],
            [170,   0,   0,   0,   0,   0,   0,   0]],
  
           [[  3,   0,   0,   0,   0,   0,   0,   0],
            [170,   0,   0,   0,   0,   0,   0,   0]]],
  
  
          [[[  3,   0,   0,   0,   0,   0,   0,   0],
            [170,   0,   0,   0,   0,   0,   0,   0]],
  
           [[170,   0,   0,   0,   0,   0,   0,   0],
            [170,   0,   0,   0,   0,   0,   0,   0]],
  
           [[180, 192,   0,   0,   0,   0,   0,   0],
            [170,   0,   0,   0,   0, 

Here I'm checking if with my "FIXED" dynamics flag (rtfm/tasks/groups.py) I'm able to keep fixed the wiki and the objects in the map, but randomize initial conditions.

In [19]:
gym_env.get_wiki()

'blessed, fanatical beat cold. grandmasters, soldiers beat fire. mysterious, shimmering beat lightning. arcane, gleaming beat poison. bat, ghost, goblin are order of the forest. imp, panther, zombie are rebel enclave. jaguar, shaman, wolf are star alliance.'

In [20]:
f = gym_env.reset()
gym_env.get_wiki()

'blessed, fanatical beat cold. grandmasters, soldiers beat fire. mysterious, shimmering beat lightning. arcane, gleaming beat poison. bat, ghost, goblin are order of the forest. imp, panther, zombie are rebel enclave. jaguar, shaman, wolf are star alliance.'

In [21]:
f['name'].shape

torch.Size([5, 5, 2, 2])

In [22]:
f['inv'].shape

torch.Size([2])

In [23]:
f['name'].shape

torch.Size([5, 5, 2, 2])

In [24]:
gym_env = create_env(flags)
gym_env.get_wiki()

'blessed, fanatical beat cold. grandmasters, soldiers beat fire. mysterious, shimmering beat lightning. arcane, gleaming beat poison. bat, ghost, goblin are order of the forest. imp, panther, zombie are rebel enclave. jaguar, shaman, wolf are star alliance.'

In [26]:
f = gym_env.reset()
f['name'][1,1,:,:]

tensor([[180,   0],
        [170,   0]])

In [27]:
for k in f.keys():
    print(k, f[k].shape)

name torch.Size([5, 5, 2, 2])
name_len torch.Size([5, 5, 2])
inv torch.Size([2])
inv_len torch.Size([1])
wiki torch.Size([80])
wiki_len torch.Size([1])
task torch.Size([40])
task_len torch.Size([1])
valid torch.Size([5])


In [39]:
f['valid'].numpy()

array([1., 1., 1., 1., 0.], dtype=float32)

In [28]:
# the underlying env takes as input simple integers in (0,4)
frame, reward, done, info = gym_env.step(0) 

In [40]:
reward

-0.02

In [29]:
done

False

In [30]:
def process_frame(frame):
    W,H,P,C = frame['name'].shape
    names_grid = frame['name'].reshape(W,H,-1)
    inv_tiled = frame['inv'].expand(W,H,-1)
    obs = torch.cat([names_grid, inv_tiled], axis=-1).reshape(W,H,-1)
    return obs.numpy()

In [31]:
obs = process_frame(f)
obs.shape

(5, 5, 6)

In [ ]:
len(gym_env.vocab)

In [ ]:
obs = torch.tensor(obs).unsqueeze(0).float()

In [ ]:
obs = obs.type(torch.int64)

In [ ]:
demb = 20
observation_space = (6,6,26)
emb = nn.Embedding(len(gym_env.vocab), demb)
conv1 = nn.Conv2d(demb*observation_space[-1], 256, 3)
conv2 = nn.Conv2d(256, 128, 1)
linear1 = nn.Linear(128*(observation_space[0]-2)**2, 256)

In [14]:
help(nn.Embedding)

Help on class Embedding in module torch.nn.modules.sparse:

class Embedding(torch.nn.modules.module.Module)
 |  Embedding(num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False, _weight=None)
 |  
 |  A simple lookup table that stores embeddings of a fixed dictionary and size.
 |  
 |  This module is often used to store word embeddings and retrieve them using indices.
 |  The input to the module is a list of indices, and the output is the corresponding
 |  word embeddings.
 |  
 |  Args:
 |      num_embeddings (int): size of the dictionary of embeddings
 |      embedding_dim (int): the size of each embedding vector
 |      padding_idx (int, optional): If given, pads the output with the embedding vector at :attr:`padding_idx`
 |                                       (initialized to zeros) whenever it encounters the index.
 |      max_norm (float, optional): If given, each embedding vector with norm larger than :attr:`max_nor

In [ ]:
x = emb(obs)
print(x.shape)
x = x.reshape(x.shape[:-2]+(-1,))
print(x.shape)
x = x.transpose(2,3).transpose(1,2)
print(x.shape)
x = F.relu(conv1(x))
print(x.shape)
x = F.relu(conv2(x))
print(x.shape)
x = x.reshape(x.shape[0],-1)
print(x.shape)
x = F.relu(linear1(x))
print(x.shape)

In [ ]:
help(x.transpose)

In [ ]:
help(torch.transpose)

In [ ]:
help(x.permute)

In [ ]:
gym_env.valid

In [ ]:
x = torch.rand((10,1))
y = torch.rand((10,10,1))

In [ ]:
x.expand(10,10,1).shape

In [ ]:
x.squeeze().shape

In [ ]:
f['name'].squeeze().shape

In [ ]:
x.expand_as(y).shape

### MCTS ideas

Some ideas on how to design a standard MCTS (no policy or value networks used) for this environment.
1. Wiki and task are fixed for an episode, so we don't consider them part of the tree since they don't change along the nodes.
2. name and inv can change, so we can use the simplest possible joint representation of the two as the node representation
3. **Not all dynamics are deterministic, since on higher difficulties (i.e. non stationary monsters), the movement pattern of the monsters seems random. This complicates things, since a state S and an edge (S,A) don't always lead to the same S' !** 
4. We can use the vector of valid actions to avoid going against the wall, but this won't be available when we will use a learned model of the environment (unless we specifically train for it). 
5. We should make a deep copy of the environment at the beginning of each search, in order to avoid changing the true state of the system while running simulations.
6. It would be a good thing to desing the interactions with the "simulator" as if it was already a learned NN and not a hardcoded simulator.
7. We should keep the part of the tree corresponding to the root node child that has been selected.
8. Ideally we will expand this class in 2 directions: to include root parallelization MCTS and to include policy and value networks (plus a custom fast rollout policy better than random).

In [37]:
class MCTS():
    def __init__(self, 
                 simulator,
                 valid_moves,
                 ucb_C,
                 discount,
                 max_actions,
                 root=None):
        """
        Monte Carlo Tree Search assuming deterministic dynamics.
        
        simulator: 
            wrapper of the environment that returns a scalar reward, a list of valid actions 
            and a 'done' boolean flag when presented with an action
        valid_moves:
            list of valid moves for the root node
        ucb_c:
            Constantused in the UCB1 formula for trees
            UCB(s,a) = Q(s,a) + ucb_c*sqrt(log N(s,a)/(\sum_b N(s,b)))
        discount:
            discoung factor gamma of the MDP
        max_actions:
            number of actions to be taken at most from the root node to the end of a rollout
        root: 
            might be the child of an old root node; use it to keep all the cached computations 
            from previous searches with a different root node. 
        """
        self.simulator = copy.deepcopy(simulator) 
        self.valid_moves = valid_moves
        self.action_space = len(valiv_moves)
        self.ucb_c = ucb_c
        self.discount = discount
        self.max_actions = max_actions
        self.root = root
    
    def get_subtree(self, action):
        """
        Returns the subtree whose root node is the current root's child corresponding to
        the given action.
        """
        return root.children[action]
    
    def run(self, num_simulations):
        """
        Runs num_simulations searches starting from the root node corresponding to the internal
        state of the simulator given during initialization.
        Returns the root node and an extra_info dictionary
        """
        if self.root is None:
            self.root = Node()
            self.root.expand(
                self.valid_actions,
                0, # reward to get to root
                self.simulator # state of the simulator at the root node 
            )
        
        max_tree_depth = 0
        root = self.root
        for n in range(num_simulations):
            ### Start of a simulation/search ###
            node = root
            search_path = [node]
            current_tree_depth = 0
            
            ### Selection phase until leaf node is reached ###
            while node.expanded() or (current_tree_depth==self.max_actions):
                current_tree_depth += 1
                action, node = self.select(node)
                if node.expanded():
                    search_path.append(node)
                
            ### Expansion of leaf node ###
            parent = search_path[-1] # last expanded node on the search path
            node = self.expand(node, parent, action)
            search_path.append(node)
            
            ### Simulation phase for self.max_actions - current_tree_depth steps ###
            value = self.simulate(node, current_tree_depth)
    
            ### Backpropagation of the leaf node value along the seach_path ###
            self.backprop(value, search_path)
        
            max_tree_depth = max(max_tree_depth, current_tree_depth)

        extra_info = {
            "max_tree_depth": max_tree_depth
        }
        # just a check to see if root works as a shallow copy of self.root
        assert root.visit_count == self.root.visit_count, "self.root not updated during search"
        
        return root, extra_info
        
    def select(self, node):
        """
        Use UCT formula on the input node; return the action selected and the corresponding child node 
        """
        
        max_ucb = max(
            self.ucb_score(node, child)
            for action, child in node.children.items()
        )
        action = numpy.random.choice(
            [
                action
                for action, child in node.children.items()
                if self.ucb_score(node, child) == max_ucb
            ]
        )
        return action, node.children[action]

    def ucb_score(self, parent, child, eps=1e-3):
        """
        The score for a node is based on its value, plus an exploration bonus.
        """
        exploration_term = self.ucb_c*np.sqrt(np.log(parent.visit_count)/(child.visit_count+eps))

        if child.visit_count > 0:
            # Mean value Q
            value_term = child.reward + self.discount*child.value() 
        else:
            value_term = 0

        return value_term + exploration_term
    
    def expand(self, node, parent, action):
        """
        Expand the node obtained by taking the given action from the parent node 
        """
        simulator = parent.get_simulator() # get a deepcopy of the simulator with the parent's state stored
        reward, valid_actions, done = simulator.step(action) # this also updates the simulator's internal state
        node.expand(valid_actions, reward, simulator)
        return node
    
    def simulate(self, node, current_depth):
        """
        Simulate a rollout with a random policy starting from the input node
        until the end of the episode or self.max_actions are reached 
        (also considering the current depth of the input node from the root)
        """
        simulator = node.get_simulator()
        valid_actions = node.valid_actions
        steps = self.max_actions - current_depth
        cum_discounted_reward = 0
        for i in range(steps):
            action = np.random.choice(valid_actions)
            reward, valid_actions, done = simulator.step(action)
            cum_discounted_reward += (self.discount**i)*reward
            if done:
                break
        return cum_discounted_reward
            
    def backprop(self, search_path, value):
        """
        Update the value sum and visit count of all nodes along the search path.
        """
        for node in reversed(search_path):
            node.value_sum += value
            node.visit_count += 1
            value = node.reward + self.discount*value



In [36]:
class Node:
    def __init__(self):
        self.visit_count = 0
        self.value_sum = 0
        self.children = {}
        self.reward = 0
        self.simulator = None
        
    def expanded(self):
        return len(self.children) > 0

    def value(self):
        if self.visit_count == 0:
            return 0
        return self.value_sum / self.visit_count

    def expand(self, valid_actions, reward, simulator):
        self.reward = reward
        for action in valid_actions:
            self.children[action] = Node()
        self.simulator = copy.deepcopy(simulator)
        
    def get_simulator(self):
        return copy.deepcopy(self.simulator)
    
    def best_action(self, discount):
        """
        Look among the children and take the one with higher Q-value. 
        Exclude children with 0 visits.
        """
        actions = []
        Qvalues = []
        for action, child in node.children.items():
            actions.append(action)
            Qvalues.append(child.reward + discount*child.value())
        actions = np.array(actions)
        Qvalues = np.array(Qvalues)
        max_Q = Qvalues.max()
        mask = (Qvalues==max_Q)
        best_actions = actions[mask]
        return np.random.choice(best_actions)